In [18]:
path_org='/Users/bidhanbashyal/MSU/Research/DataAug4SocialBias/SentenceGeneration/Data/DebiasingCorpus/corpus(10-40)10k.txt'

In [64]:
path_sim='/Users/bidhanbashyal/MSU/Research/DataAug4SocialBias/Notebook/output_generated_sentences_100_low_ttr.txt'

In [65]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # Downloading necessary NLTK data

def calculate_total_tokens_and_ngrams(sentences):
    total_tokens = 0
    unique_tokens = set()

    for sentence in sentences:
        # Tokenizing the sentence
        tokens = word_tokenize(sentence)
        total_tokens += len(tokens)

        # Updating the set of unique tokens
        unique_tokens.update(tokens)

    total_unique_tokens = len(unique_tokens)

    return total_tokens/len(sentences), total_unique_tokens/len(sentences)





[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bidhanbashyal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
same_sentences = []
with open(path_sim, 'r', encoding='utf-8') as file:
    for line in file:
        # Strip to remove any leading/trailing whitespace
        sentence = line.strip()
        # Only add non-empty sentences
        if sentence:
            same_sentences.append(sentence)


In [67]:
same_sentences

['The trump administration’s red line was apparently crossed this month, when a white house spokesman issued a rare rebuke of netanyahu, dismissing reports that us officials had discussed an annexation plan for the west bank with their israeli counterparts, this move has strained the relations between the two allies. Adding to the already tense situation between the two nations.',
 "In California, for example, Governor Gavin Newsom (whom Lenny Mendonca advises) is developing a comprehensive strategy to upskill the state’s diverse workforce, catering to the needs of the state's diverse population and workforce.",
 "We don't often hear about the successful organization of small-scale local elections and workshops for women's empowerment, yet they are essential for the development of the country.",
 "Whatever emerges from his next summit with Kim, it is already clear that Trump's disregard for US alliances is taking a toll, causing frustration for US allies who fear his impulsive decision

In [68]:
org_sentences = []
with open(path_org, 'r', encoding='utf-8') as file:
    for line in file:
        # Strip to remove any leading/trailing whitespace
        sentence = line.strip()
        # Only add non-empty sentences
        if sentence:
            org_sentences.append(sentence)

In [69]:
org_sentences=org_sentences[:100]
n=1

## Checking number of tokens

In [70]:
average_tokens, total_bigrams =  calculate_total_tokens_and_ngrams(same_sentences)
print(f"Average number of tokens per sentence: {average_tokens}")
print(f"Total number of grams in the tokens: {total_bigrams}")

Average number of tokens per sentence: 42.53
Total number of grams in the tokens: 15.47


In [71]:
average_tokens, total_bigrams =  calculate_total_tokens_and_ngrams(org_sentences)
print(f"Average number of tokens per sentence: {average_tokens}")
print(f"Total number of {n}-grams in the tokens: {total_bigrams}")

Average number of tokens per sentence: 29.25
Total number of 1-grams in the tokens: 12.22


In [72]:
len(org_sentences)

100

## Checking similarity

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


vectorizer = TfidfVectorizer()
# Calculate similarity for each pair of sentences
for i in range(min(len(org_sentences), len(same_sentences))):
    # Vectorize the sentences
    tfidf_matrix = vectorizer.fit_transform([org_sentences[i], same_sentences[i]])

    # Calculate the cosine similarity
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    print(f"Similarity of sentence {i+1}: {similarity[0][0]}")

Similarity of sentence 1: 0.727797450372317
Similarity of sentence 2: 0.7702343747290071
Similarity of sentence 3: 0.7494852396814299
Similarity of sentence 4: 0.7396162596206347
Similarity of sentence 5: 0.7287472757273957
Similarity of sentence 6: 0.8519476944064879
Similarity of sentence 7: 0.7696713426132546
Similarity of sentence 8: 0.7376850077080517
Similarity of sentence 9: 0.5468888321391275
Similarity of sentence 10: 0.786814852401053
Similarity of sentence 11: 0.8182780611358194
Similarity of sentence 12: 0.583782790261462
Similarity of sentence 13: 0.8332787721241084
Similarity of sentence 14: 0.7065966901282225
Similarity of sentence 15: 0.9136969907736722
Similarity of sentence 16: 0.7099353880051468
Similarity of sentence 17: 0.44180801288095956
Similarity of sentence 18: 0.8986267513424608
Similarity of sentence 19: 0.245258208932404
Similarity of sentence 20: 0.6692838535697996
Similarity of sentence 21: 0.3947522034732363
Similarity of sentence 22: 0.6853880419233512


## Checking text token ratio

In [74]:
def calculate_ttr(sentences_list):
    def get_word_tokens(sentences):
        tokens = []
        for sentence in sentences:
            # Split each sentence into words
            words = sentence.split()
            tokens.extend(words)
        return tokens

    # Get word tokens from the given list
    tokens = get_word_tokens(sentences_list)

    # Calculate the number of types (unique words)
    types = len(set(tokens))

    # Calculate the number of tokens (total words)
    tokens_total = len(tokens)

    # Calculate the type-token ratio (TTR)
    ttr = types / tokens_total if tokens_total > 0 else 0

    return ttr

In [75]:
ratio_list1 = calculate_ttr(org_sentences)
ratio_list2 = calculate_ttr(same_sentences)

print(f"Text-to-token ratio for list1: {ratio_list1:.2f}")
print(f"Text-to-token ratio for list2: {ratio_list2:.2f}")


Text-to-token ratio for list1: 0.51
Text-to-token ratio for list2: 0.45


In [76]:
def calculate_hapax_legomenon_ratio(sentences_list):
    def get_word_tokens(sentences):
        tokens = []
        for sentence in sentences:
            # Split each sentence into words
            words = sentence.split()
            tokens.extend(words)
        return tokens

    # Get word tokens from the given list
    tokens = get_word_tokens(sentences_list)

    # Calculate the number of hapax legomena (words occurring only once)
    hapax_legomena = [word for word in set(tokens) if tokens.count(word) == 1]

    # Calculate the hapax legomenon ratio (HLR)
    hlr = len(hapax_legomena) / len(tokens) if len(tokens) > 0 else 0

    return hlr


In [77]:
ratio_list1 = calculate_hapax_legomenon_ratio(org_sentences)
ratio_list2 = calculate_hapax_legomenon_ratio(same_sentences)

print(f"Text-to-token ratio for list1: {ratio_list1:.2f}")
print(f"Text-to-token ratio for list2: {ratio_list2:.2f}")


Text-to-token ratio for list1: 0.41
Text-to-token ratio for list2: 0.35


## Readability

In [61]:
import textstat

def calculate_readability_scores(sentences):
    # Join the list of sentences into a single text
    text = ' '.join(sentences)

    # Calculate readability scores
    flesch_kincaid = textstat.flesch_kincaid_grade(text)
    gunning_fog = textstat.gunning_fog(text)

    return flesch_kincaid, gunning_fog


In [62]:
flesch_kincaid_score, gunning_fog_score = calculate_readability_scores(same_sentences)

print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_score}")
print(f"Gunning Fog Index: {gunning_fog_score}")


Flesch-Kincaid Grade Level: 17.4
Gunning Fog Index: 19.71


In [63]:
flesch_kincaid_score, gunning_fog_score = calculate_readability_scores(org_sentences)

print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_score}")
print(f"Gunning Fog Index: {gunning_fog_score}")

Flesch-Kincaid Grade Level: 12.9
Gunning Fog Index: 15.56
